## DeepLearning model API with docker container

After saving the model (we'll assume that we already have the model in a folder with its folder name is the model name), we will create an API using the following steps:

- Install Docker (link to download : https://docs.docker.com/desktop/install/windows-install/)
- Install wsl2 (link to download : https://docs.microsoft.com/en-us/windows/wsl/install-manual#step-4---download-the-linux-kernel-update-package)
- Run Docker
- Open terminal / cmd and type :
```bash
docker pull tensorflow/serving
docker run --rm -it -p 8500:8500 -p 8501:8501 -v "/path/to/model/traffic_classifier:/models/traffic_classifier"-e MODEL_NAME=traffic_classifier tensorflow/serving:latest
```


**Notes:** <br>
*/path/to/model/traffic_classifier* is the path where we save the model. You can download my [saved model](https://github.com/marwanmusa/German-TrafficSign-Classification-CNN/tree/main/traffic_classifier) and saved it in your PC/Notebook.

*MODEL_NAME* is our model name.

if it success, then it will writes "entering loop" on the terminal.

***This is how our API will look like***<br>
http://localhost:8501/v1/models/traffic_classifier:predict


---

# ***Hands On***

# Import Libraries

In [1]:
# Fundamental classes
import numpy as np 
import pandas as pd 
import tensorflow as tf
import os

# Image related
import cv2
from PIL import Image
import pydot

# For ploting
import matplotlib.pyplot as plt

# For the model and it's training
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [2]:
dataset_directory = "/path/to/model_folder"
train_path = "/path/to/model_folder/Train"

In [3]:
# Dictionary of each class
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

# Model Inference

Finally, let's use our model to classify an image that wasn't included in the training or validation sets.

In [4]:
# Loading data inference
test_image = dataset_directory + r'/Test/00057.png'

img = keras.preprocessing.image.load_img(
    test_image, target_size=([30, 30])
)

img # showing image

In [5]:
# predicting with docker API
import requests
import json
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

# Processing
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

# json input
input_data_json_img = json.dumps({
    "signature_name": "serving_default",
    "instances": img_array.tolist(),
})

# predicting
URL = 'http://localhost:8501/v1/models/traffic_classifier:predict'
response_img = requests.post(URL, data=input_data_json_img)
response_img.raise_for_status() # raise an exception in case of error
response_img = response_img.json()

# predicting result
score_img = tf.nn.softmax(response_img['predictions'][0])
destination_by_img = classes[np.argmax(score_img)]

# predicting result
score = tf.nn.softmax(response_img['predictions'][0])
print(
    "### This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(classes[np.argmax(score)], 100 * np.max(score))
)

### This image most likely belongs to Traffic signals with a 6.08 percent confidence.


In [9]:
score_img # what's in score_img

<tf.Tensor: shape=(43,), dtype=float32, numpy=
array([0.02236222, 0.02236222, 0.02236222, 0.02236222, 0.02236222,
       0.02236222, 0.02236222, 0.02236222, 0.02236222, 0.02236222,
       0.02236222, 0.02236222, 0.02236222, 0.02236222, 0.02236222,
       0.02236222, 0.02236222, 0.02236222, 0.02236222, 0.02236222,
       0.02236222, 0.02236222, 0.02236222, 0.02236222, 0.02236222,
       0.02236222, 0.06078682, 0.02236222, 0.02236222, 0.02236222,
       0.02236222, 0.02236222, 0.02236222, 0.02236222, 0.02236222,
       0.02236222, 0.02236222, 0.02236222, 0.02236222, 0.02236222,
       0.02236222, 0.02236222, 0.02236222], dtype=float32)>

In [6]:
# custom method for generating predictions
def getPredictions(file_url):
     # Fundamental classes
        import numpy as np 
        import pandas as pd 
        import tensorflow
        import os

        # Image related
        import cv2
        from PIL import Image
        import pydot

        # For ploting
        import matplotlib.pyplot as plt

        # For the model and it's training
        from sklearn.metrics import accuracy_score
        from sklearn.model_selection import train_test_split
        from tensorflow import keras
        from tensorflow.keras.utils import to_categorical, plot_model
        from tensorflow.keras.models import Sequential, load_model
        from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

        # predicting with docker API
        import requests
        import json
        from tensorflow.python.ops.numpy_ops import np_config
        np_config.enable_numpy_behavior()
        #
        # Django image API
        #

        #
        # classes
        # Dictionary of each class
        classes = { 0:'Speed limit (20km/h)',
                    1:'Speed limit (30km/h)', 
                    2:'Speed limit (50km/h)', 
                    3:'Speed limit (60km/h)', 
                    4:'Speed limit (70km/h)', 
                    5:'Speed limit (80km/h)', 
                    6:'End of speed limit (80km/h)', 
                    7:'Speed limit (100km/h)', 
                    8:'Speed limit (120km/h)', 
                    9:'No passing', 
                    10:'No passing veh over 3.5 tons', 
                    11:'Right-of-way at intersection', 
                    12:'Priority road', 
                    13:'Yield', 
                    14:'Stop', 
                    15:'No vehicles', 
                    16:'Veh > 3.5 tons prohibited', 
                    17:'No entry', 
                    18:'General caution', 
                    19:'Dangerous curve left', 
                    20:'Dangerous curve right', 
                    21:'Double curve', 
                    22:'Bumpy road', 
                    23:'Slippery road', 
                    24:'Road narrows on the right', 
                    25:'Road work', 
                    26:'Traffic signals', 
                    27:'Pedestrians', 
                    28:'Children crossing', 
                    29:'Bicycles crossing', 
                    30:'Beware of ice/snow',
                    31:'Wild animals crossing', 
                    32:'End speed + passing limits', 
                    33:'Turn right ahead', 
                    34:'Turn left ahead', 
                    35:'Ahead only', 
                    36:'Go straight or right', 
                    37:'Go straight or left', 
                    38:'Keep right', 
                    39:'Keep left', 
                    40:'Roundabout mandatory', 
                    41:'End of no passing', 
                    42:'End no passing veh > 3.5 tons' }
        # loading
        img = keras.preprocessing.image.load_img(file_url, target_size=([30, 30]))

        # Processing
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = tensorflow.expand_dims(img_array, 0)

        # json input
        input_data_json_img = json.dumps({
            "signature_name": "serving_default",
            "instances": img_array.tolist(),
        })

        # predicting
        # URL = 'https://traffic-classifier-backend.herokuapp.com/v1/models/traffic_classifier:predict'
        URL = 'http://localhost:8501/v1/models/traffic_classifier:predict'
        response_img = requests.post(URL, data=input_data_json_img)
        response_img.raise_for_status() # raise an exception in case of error
        response_img = response_img.json()

        # predicting result
        score_img = tensorflow.nn.softmax(response_img['predictions'][0])
        destination_by_img = classes[np.argmax(score_img)]

        # predicting result
        score = tensorflow.nn.softmax(response_img['predictions'][0])
        hbc = print(
            "### This image most likely belongs to {} with a {:.2f} percent confidence."
            .format(classes[np.argmax(score)], 100 * np.max(score))
        )
        return hbc

In [7]:
getPredictions(test_image)

### This image most likely belongs to Traffic signals with a 6.08 percent confidence.
